# End User's Notebook

Table of contents 
* 1\. Initialize connection to datalake using service principal name (spn)
* 2\. Read data into dataframe 
    * 2.1 case of parquet file 
    * 2.2 case of csv file : 
      * 2.2.1 without schema 
      * 2.2.2 with schema 
    * 2.3 case of delta file

1. Initialize connection to datalake using service principal name (spn)

Secret's parameters are taken from scope-datalake (key vault). 
Here are the keys:
+ client id     : sp-adbw-st-datalake-clientid
+ client secret : secret sp-adbw-st-datalake-secret
+ tenant        : sp-adbw-st-datalake-tenantid

In [0]:
storage_account_name = 'stdatalake01dev' 

parquet_file_base_path = f"abfss://bronze@{storage_account_name}.dfs.core.windows.net/QAD_PARQUET/bom_mstr/"
parquet_file_path      = parquet_file_base_path + "year=*/month=*/day=*/time=*/*.parquet"

csv_file_base_path = f"abfss://bronze@{storage_account_name}.dfs.core.windows.net/QAD/"
csv_file_path      = csv_file_base_path + "bom_mstr/year=*/month=*/day=*/time=*/*.csv"

In [0]:
spark.conf.set(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account_name}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account_name}.dfs.core.windows.net", dbutils.secrets.get(scope="scope-datalake",key="sp-adbw-st-datalake-clientid"))
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account_name}.dfs.core.windows.net", dbutils.secrets.get(scope="scope-datalake", key="sp-adbw-st-datalake-secret"))
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account_name}.dfs.core.windows.net", f"https://login.microsoftonline.com/{dbutils.secrets.get(scope='scope-datalake',key='sp-adbw-st-datalake-tenantid')}/oauth2/token")

2\. Read data into dataframe

2.1 case of parquet file

In [0]:
df = spark.read.format("parquet").option('basePath', parquet_file_base_path).load(parquet_file_path)
display(df)

In [0]:
df.schema.json()

2.2 case of csv file

In [0]:
# define df_reader 

In [0]:
df_reader = (
  spark.read.format("csv")
            .option("basePath", csv_file_base_path)
            .option("multiLine", True)
            .option('header', True)
            .option('delimiter', ";")
            .option('encoding', "UTF-8") 
)

2.2.1 without schema

In [0]:
df = (
      df_reader.option("inferSchema", True)
                .load(csv_file_path) 
)

In [0]:
display(df)

2.2.2 with schema

In [0]:
# Two ways to define schema 

In [0]:
# first way 
from pyspark.sql.types import *
schema_type = StructType([
  StructField("bom_parent",StringType(),True),
  StructField("bom_desc",StringType(),True),
  StructField("bom_batch",StringType(),True),
  StructField("bom_batch_um",StringType(),True),
  StructField("bom_cmtindx",IntegerType(),True),
  StructField("bom_ll_code",IntegerType(),True),
  StructField("bom_user1",StringType(),True),
  StructField("bom_user2",StringType(),True),
  StructField("bom_userid",StringType(),True),
  StructField("bom_mod_date",TimestampType(),True),
  StructField("bom__chr01",StringType(),True),
  StructField("bom__chr02",StringType(),True),
  StructField("bom__chr03",StringType(),True),
  StructField("bom__chr04",StringType(),True),
  StructField("bom__chr05",StringType(),True),
  StructField("bom__dte01",TimestampType(),True),
  StructField("bom__dte02",TimestampType(),True),
  StructField("bom__dec01",StringType(),True),
  StructField("bom__dec02",StringType(),True),
  StructField("bom__log01",BooleanType(),True),
  StructField("bom_formula",BooleanType(),True),
  StructField("bom_mthd",StringType(),True),
  StructField("bom_fsm_type",StringType(),True),
  StructField("bom_site",StringType(),True),
  StructField("bom_loc",StringType(),True),
  StructField("bom__qadc01",StringType(),True),
  StructField("bom__qadc02",StringType(),True),
  StructField("bom__qadc03",StringType(),True),
  StructField("bom__qadd01",StringType(),True),
  StructField("bom__qadi01",IntegerType(),True),
  StructField("bom__qadi02",IntegerType(),True),
  StructField("bom__qadt01",TimestampType(),True),
  StructField("bom__qadt02",TimestampType(),True),
  StructField("bom__qadl01",BooleanType(),True),
  StructField("bom__qadl02",BooleanType(),True),
  StructField("bom_mthd_qtycompl",StringType(),True),
  StructField("bom_domain",StringType(),True),
  StructField("oid_bom_mstr",StringType(),True),
  StructField("year",IntegerType(),True),
  StructField("month",IntegerType(),True),
  StructField("day",IntegerType(),True),
  StructField("time",IntegerType(),True)
]
)

# second way 
import json
schema_json = '{"fields":[{"metadata":{},"name":"bom_parent","nullable":true,"type":"string"},{"metadata":{},"name":"bom_desc","nullable":true,"type":"string"},{"metadata":{},"name":"bom_batch","nullable":true,"type":"string"},{"metadata":{},"name":"bom_batch_um","nullable":true,"type":"string"},{"metadata":{},"name":"bom_cmtindx","nullable":true,"type":"integer"},{"metadata":{},"name":"bom_ll_code","nullable":true,"type":"integer"},{"metadata":{},"name":"bom_user1","nullable":true,"type":"string"},{"metadata":{},"name":"bom_user2","nullable":true,"type":"string"},{"metadata":{},"name":"bom_userid","nullable":true,"type":"string"},{"metadata":{},"name":"bom_mod_date","nullable":true,"type":"timestamp"},{"metadata":{},"name":"bom__chr01","nullable":true,"type":"string"},{"metadata":{},"name":"bom__chr02","nullable":true,"type":"string"},{"metadata":{},"name":"bom__chr03","nullable":true,"type":"string"},{"metadata":{},"name":"bom__chr04","nullable":true,"type":"string"},{"metadata":{},"name":"bom__chr05","nullable":true,"type":"string"},{"metadata":{},"name":"bom__dte01","nullable":true,"type":"timestamp"},{"metadata":{},"name":"bom__dte02","nullable":true,"type":"timestamp"},{"metadata":{},"name":"bom__dec01","nullable":true,"type":"string"},{"metadata":{},"name":"bom__dec02","nullable":true,"type":"string"},{"metadata":{},"name":"bom__log01","nullable":true,"type":"boolean"},{"metadata":{},"name":"bom_formula","nullable":true,"type":"boolean"},{"metadata":{},"name":"bom_mthd","nullable":true,"type":"string"},{"metadata":{},"name":"bom_fsm_type","nullable":true,"type":"string"},{"metadata":{},"name":"bom_site","nullable":true,"type":"string"},{"metadata":{},"name":"bom_loc","nullable":true,"type":"string"},{"metadata":{},"name":"bom__qadc01","nullable":true,"type":"string"},{"metadata":{},"name":"bom__qadc02","nullable":true,"type":"string"},{"metadata":{},"name":"bom__qadc03","nullable":true,"type":"string"},{"metadata":{},"name":"bom__qadd01","nullable":true,"type":"string"},{"metadata":{},"name":"bom__qadi01","nullable":true,"type":"integer"},{"metadata":{},"name":"bom__qadi02","nullable":true,"type":"integer"},{"metadata":{},"name":"bom__qadt01","nullable":true,"type":"timestamp"},{"metadata":{},"name":"bom__qadt02","nullable":true,"type":"timestamp"},{"metadata":{},"name":"bom__qadl01","nullable":true,"type":"boolean"},{"metadata":{},"name":"bom__qadl02","nullable":true,"type":"boolean"},{"metadata":{},"name":"bom_mthd_qtycompl","nullable":true,"type":"string"},{"metadata":{},"name":"bom_domain","nullable":true,"type":"string"},{"metadata":{},"name":"oid_bom_mstr","nullable":true,"type":"string"},{"metadata":{},"name":"year","nullable":true,"type":"integer"},{"metadata":{},"name":"month","nullable":true,"type":"integer"},{"metadata":{},"name":"day","nullable":true,"type":"integer"},{"metadata":{},"name":"time","nullable":true,"type":"integer"}],"type":"struct"}'

schema_type_from_json = StructType.fromJson(json.loads(schema_json))


In [0]:
# Test if both schema are equivalent ?
schema_type_from_json == schema_type # True

In [0]:

df = (
      df_reader.schema(schema_type)
                .load(csv_file_path) 
)
display(df)


2.2 case of delta file

In [0]:
df = spark.read.format('delta').load("abfss://silver@stdatalake01dev.dfs.core.windows.net/QAD/bom_mstr/")
display(df)